# REINFORCE 구현

## `Gymnaisum` 사용법

먼저 강화학습 코딩이 처음인 분들에게는 에이전트가 환경과 상호작용하는 것부터 어떻게 구현해야 할지 막막할 것이다. `Gymnaisum`은 다양한 연습용 환경을 제공하고 있으며, 강화학습 연구자들에게 환경을 구현하기 위한 일종의 약속 (convention)을 제공하고 있다. 이번 절에서 `Gymnaisum`에서 제공하고 있는 CartPole 환경을 사용해볼 것이다.

<br>

먼저 실습에 사용할 라이브러리들을 모두 임포트 하자.


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gymnasium as gym
import matplotlib.pyplot as plt

<br>

`gymnasium.make('환경이름')`을 통해 환경을 불러올 수 있다. `Gymnasium`에서 제공하는 환경의 목록은 `Gymnasium`의 [공식 문서](https://gymnasium.farama.org/environments/classic_control/)에 잘 나와 있다.

In [2]:
env = gym.make('CartPole-v1')

<br>

환경 초기화는 `env.reset()` 메서드를 사용하여 할 수 있다. 초기 상태 $s_0 \sim \rho_0$에 대응하는 코드이다. 이 메서드를 호출하면 환경의 초기 상태와 주로 학습에 사용되지는 않지만 사용자가 알면 좋은 추가 정보를 반환해준다.


In [3]:
s, info = env.reset()

print("Initial state is: ", s)
print("Information: ", info)

Initial state is:  [ 0.04483376 -0.00665065 -0.0137544  -0.02688403]
Information:  {}


<br>

환경의 상태를 관찰하였으니, 아무 행동을 뽑아서 환경에 행동을 취해보자. `env.action_space.sample()`은 행동 공간에서 임의의 행동 하나를 반환해주는 메서드이다. 다음으로 `env.step(action)`을 통해 환경에 행동을 취할 수 있다. `env.step(action)`은 5-tuple인 `(s_prime, r, terminated, truncated, info)`을 반환해준다.
- `s_prime`: 다음 상태
- `r`: 보상
- `terminated`: 환경이 종료 조건에 의해 종료되었는지 여부
- `truncated`: 환경이 최대 상호작용 횟수에 도달하여 종료되었는지 여부
- `info`: 추가 정보


In [4]:
# Choose a random action
a = env.action_space.sample()  

# Take the action
s_prime, r, terminated, truncated, info =  env.step(a)

print("Next state: ", s_prime)
print("Reward: ", r)
print("Is terminated? ", terminated)
print("Is truncated? ", truncated)
print("Information: ", info)

Next state:  [ 0.04470074 -0.20157267 -0.01429208  0.2614277 ]
Reward:  1.0
Is terminated?  False
Is truncated?  False
Information:  {}


```{note}
`terminated`와 `truncated`은 둘 다 환경이 종료되었는지 여부를 반환해준다. 단, `terminated`은 종료 조건에 의해 환경이 종료되었을 때 `True`를 반환한다. 종료 조건의 예시는 주로 에이전트가 목적을 달성했거나, 사망이나 붕괴 등 돌이킬 수 없는 상태에 빠졌을 경우를 의미한다. `truncated`은 주로 환경의 종료 조건이 따로 없는 경우에만 의미가 있다. 로봇 통제와 같은 무한히 상호작용할 수 있는 환경의 경우, 실제 구현에서는 평생 상호작용할 수 없으니 1000회 정도 상호작용 후 환경을 마친다. 이처럼 최대 상호작용 횟수에 도달하여 환경이 종료된 경우 `truncated`와 `True`를 반환한다 
```

<br>

위의 내용을 종합하면, 한 에피소드 진행은 다음과 같이 구현한다.

In [5]:
s, terminated, truncated, ret = env.reset(), False, False, 0
while not (terminated or truncated):
    a = env.action_space.sample()
    s_prime, r, terminated, truncated, _ = env.step(a)
    
    ret += r
    s = s_prime
print("Return: ", ret)

Return:  29.0


<br>

---

## 이산 행동 공간

다음으로 정책 네트워크를 정의해보자. 정책은 주어진 상태 $s$에서 행동 $a$를 취한 조건부 확률 $\text{Pr}\left[A=a|S=s \right]$이다. 
각 상태 $s$마다 조건부 확률 분포가 정의되어야 하므로 정책 네트워크는 상태를 입력 받는다. 그리고 행동에 대한 확률 분포이기 때문에 각 행동에 확률을 부여해야 한다.

CartPole 환경은 4차원 행동을 입력 받고 2가지 행동 중 선택하는 이산 행동 공간을 가지기 때문에 4차원 벡터를 입력 받고 2차원 벡터를 출력하는 다층퍼셉트론 (MLP)로 정의하되, 출력이 확률 분포로 해석할 수 있도록 출력에 softmax 함수를 씌울 것이다. 이 MLP 나중에 다양한 실험을 위해서 다소 일반적으로 코드를 짰다. 일반적이라는 의미는 사용자의 수요에 맞게 히든레이어 개수 및 노드 수를 조정할 수 있고 활성화 함수도 설정할 수 있도록 만들었다.

In [6]:
class MLPDiscretePolicy(nn.Module):
    def __init__(self, dim_state, dim_action, dim_hiddens=(512, ), activation_fn=F.relu):
        super(MLPDiscretePolicy, self).__init__()
        self.input_layer = nn.Linear(dim_state, dim_hiddens[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(dim_hiddens) - 1):
            hidden_layer = nn.Linear(dim_hiddens[i], dim_hiddens[i+1])
            self.hidden_layers.append(hidden_layer)
        self.output_layer = nn.Linear(dim_hiddens[-1], dim_action)
        self.activation_fn = activation_fn

    def forward(self, s):
        s = self.activation_fn(self.input_layer(s))
        for hidden_layer in self.hidden_layers:
            s = self.activation_fn(hidden_layer(s))
        prob = F.softmax(self.output_layer(s), dim=-1)

        return prob

<br>

정책 네트워크를 다음과 같이 선언할 수 있다.

In [7]:
env = gym.make('CartPole-v1')

dim_state = env.observation_space.shape[0]
dim_action = env.action_space.n
dim_hiddens = (128, 128)
activation_fn = F.relu

policy = MLPDiscretePolicy(dim_state, dim_action, dim_hiddens, activation_fn)
policy

MLPDiscretePolicy(
  (input_layer): Linear(in_features=4, out_features=128, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=128, out_features=128, bias=True)
  )
  (output_layer): Linear(in_features=128, out_features=2, bias=True)
)

<br>

정책 네트워크에 상태 하나를 입력하면 다음과 같이 출력된다. 결과로 나온 벡터의 각 원소는 각 행동을 취할 확률로 해석할 수 있다.

In [8]:
s, _ = env.reset()
s = torch.as_tensor(s, dtype=torch.float)

policy(s)

tensor([0.4787, 0.5213], grad_fn=<SoftmaxBackward0>)

<br>

정책 네트워크로 만든 확률 분포에서 행동을 샘플링은 다음과 같이 할 수 있다.

In [9]:
prob = policy(s)
a = torch.multinomial(prob, num_samples=1)
print("Selected action : ", a.item())

Selected action :  1


<br>

정책 네트워크가 준비되었으니 REINFORCE 에이전트를 만들어보자. REINFORCE 에이전트는 총 4개 메서드가 있다. 
- `__init__()`: 에이전트 클래스가 입력 받을 하이퍼파라미터를 입력 받고 여러가지 초기화를 수행한다.
- `act()`: 환경과의 상호작용을 위한 메서드로서 상태 하나를 입력 받아 행동을 출력한다.
- `learn()`: 에이전트가 갖고 있는 네트워크들을 업데이트해준다.
- `process()`: 환경과의 매 상호작용마다 수행할 메서드이다. 주로 transition을 버퍼에 저장하고, 특정 주기로 `learn()` 메서드를 호출한다.

이 메서드 구성은 카카오 엔터프라이즈의 강화학습 라이브러리 [JORLDY](https://github.com/kakaoenterprise/JORLDY)를 참고하여 만들었다. 이후 구현할 알고리즘들도 동일한 메서드 구성을 갖고 있다. 알고리즘마다 각 메서드가 어떻게 구현되는지에 초점을 맞춰 비교하면 좋을 것 같다.

In [10]:
class REINFORCE:
    def __init__(self, policy, gamma=0.99, lr=0.001):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.policy = policy.to(self.device)
        self.gamma = gamma
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr)
        self.buffer = []
        
    @torch.no_grad()
    def act(self, s, training=True):
        self.policy.train(training)

        s = torch.as_tensor(s, dtype=torch.float, device=self.device)
        prob = self.policy(s)
        a = torch.multinomial(prob, 1) if training else torch.argmax(prob, dim=-1, keepdim=True)

        return a.cpu().numpy()

    def learn(self):
        self.policy.train()
        s, a, r, _, _, _ = map(np.stack, zip(*self.buffer))
        s, a, r = map(lambda x: torch.as_tensor(x, dtype=torch.float, device=self.device), [s, a, r])
        a = a.long()
        r = r.unsqueeze(1)
        
        ret = torch.clone(r)
        for t in reversed(range(len(ret) - 1)):
            ret[t] += self.gamma * ret[t + 1]
            
        probs = self.policy(s)
        log_probs = torch.log(probs.gather(1, a.long()))
        
        policy_loss = - (ret * log_probs).mean()
        self.optimizer.zero_grad()
        policy_loss.backward()
        self.optimizer.step()
        
        result = {'policy_loss': policy_loss.item()}
        
        return result
        
    def process(self, transition):
        result = None
        self.buffer.append(transition)
        if transition[-1] or transition[-2]:
            result = self.learn()
            self.buffer = []
        return result

<br>

REINFORCE 클래스 작성을 마쳤으니 이제 환경과 상호작용을 하면서 정책 네트워크를 훈련시킬 차례이다.

In [11]:
def evaluate(env_name, agent, eval_iterations):
    env = gym.make(env_name)
    scores = []
    for _ in range(eval_iterations):
        (s, _), terminated, truncated, score = env.reset(seed=np.random.randint(10000)), False, False, 0
        while not (terminated or truncated):
            a = agent.act(s, training=False)
            s_prime, r, terminated, truncated, _ = env.step(a[0])
            score += r
            s = s_prime
        scores.append(score)
    env.close()
    return round(np.mean(scores), 4)

In [12]:
env_name = 'CartPole-v1'

seed = 0
max_iterations = 100000
eval_intervals = 10000
eval_iterations = 10
gamma = 0.99
lr = 0.001

env = gym.make(env_name)
policy = MLPDiscretePolicy(dim_state, dim_action)
agent = REINFORCE(policy, gamma, lr)

(s, _), terminated, truncated = env.reset(), False, False
for t in range(1, max_iterations + 1):
    a = agent.act(s)
    s_prime, r, terminated, truncated, _ = env.step(a[0])
    result = agent.process((s, a, r, s_prime, terminated, truncated))
    s = s_prime
    
    if terminated or truncated:
        (s, _), terminated, truncated = env.reset(), False, False
        
    if t % eval_intervals == 0:
        score = evaluate(env_name, agent, eval_iterations)
        print(f"[Steps {t:6}] Avg return: {score:.5}")

[Steps  10000] Avg return: 477.5
[Steps  20000] Avg return: 263.2
[Steps  30000] Avg return: 333.7
[Steps  40000] Avg return: 500.0
[Steps  50000] Avg return: 500.0
[Steps  60000] Avg return: 500.0
[Steps  70000] Avg return: 500.0
[Steps  80000] Avg return: 500.0
[Steps  90000] Avg return: 153.1
[Steps 100000] Avg return: 500.0


<br>

---

## 연속 행동 공간

Coming soon!